## Lab - EDA Bivariate Analysis: Diving into Amazon UK Product Insights Part II

Objective: Delve into the dynamics of product pricing on Amazon UK to uncover insights that can inform business strategies and decision-making.

Dataset: This lab utilizes the Amazon UK product dataset which provides information on product categories, brands, prices, ratings, and more from from Amazon UK. You'll need to download it to start working with it.

# Part 1: Analyzing Best-Seller Trends Across Product Categories

Objective: Understand the relationship between product categories and their best-seller status.

1. Crosstab Analysis:
- Create a crosstab between the product category and the isBestSeller status.
- Are there categories where being a best-seller is more prevalent?
- Hint: one option is to calculate the proportion of best-sellers for each category and then sort the categories based on this proportion in descending order.

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

%matplotlib inline

In [2]:
df=pd.read_csv("amz_uk_price_prediction_dataset.csv")
df.head()

FileNotFoundError: [Errno 2] No such file or directory: 'amz_uk_price_prediction_dataset.csv'

In [ ]:
crosstab= pd.crosstab(df['category'], df['isBestSeller'])

crosstab

In [ ]:
corsstab_sorted=crosstab.sort_values(by=True,ascending=False)
print(corsstab_sorted)

print("En ninguna categoría hay un full de bestsellers o prevalece sobre el resto, pero la que más tiene es Grocery, seguida de Health & Personal Care")

2. Statistical Tests:
- Conduct a Chi-square test to determine if the best-seller distribution is independent of the product category.
- Compute Cramér's V to understand the strength of association between best-seller status and category.

In [ ]:
from scipy.stats import chi2_contingency

# Chi-square test for 'category' and 'isBestSeller'
chi2_statistic, chi2_p_value, _, _ = chi2_contingency(crosstab)

chi2_statistic, chi2_p_value

In [ ]:
#H0 -> las variables son independientes 
#H1 -> las variables son dependientes 
# p>0.05 rechazamos H0
# p>=0.05 aceptamos H0

print("""En este caso las variables son dependientes porque rechazamos H0 porque el p values es 0,0, es decir, p<0.05.
Al ser dependientes, ahora vamos a ver su intensidad en cuanto a su relación con Cramers.""")

In [ ]:
from scipy.stats.contingency import association

# Computing the association between variables in 'crosstab_result' using the "cramer" method
association(crosstab, method="cramer")

El coeficiente 0,12 sugiere una relación pobre, vaga entre las dos variables.

3. Visualizations:
- Visualize the relationship between product categories and the best-seller status using a stacked bar chart.


In [ ]:
#hay muchas categorías por loq ue el eje x es infinito e ilegible , hay que acortar el crosstab. 

top_categories=corsstab_sorted.head(10)

top_categories

In [ ]:
top_categories.plot(kind="bar", stacked=True, figsize=(10, 6), colormap='Set2')
plt.title("Best Seller Status por Categoría de Producto")
plt.xlabel("Categoría")
plt.ylabel("Número de Productos")
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.show()

 ## Part 2: Exploring Product Prices and Ratings Across Categories and Brands


Objective: Investigate how different product categories influence product prices.

Preliminary Step: Remove outliers in product prices.

For this purpose, we can use the IQR (Interquartile Range) method. Products priced below the first quartile minus 1.5 times the IQR or above the third quartile plus 1.5 times the IQR will be considered outliers and removed from the dataset. The next steps will be done with the dataframe without outliers.

Hint: you can check the last Check For Understanding at the end of the lesson EDA Bivariate Analysis for a hint on how to do this.

In [ ]:
# Eliminar registros con precio nulo (si los hubiera)
df = df[df['price'].notna()]

In [ ]:
# Calcular Q1, Q3 e IQR
Q1 = df['price'].quantile(0.25)
Q3 = df['price'].quantile(0.75)
IQR = Q3 - Q1
IQR

In [ ]:
# Definir límites
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

In [ ]:
# Filtrar el DataFrame para quitar los outliers
df_no_outliers = df[(df['price'] >= lower_bound) & (df['price'] <= upper_bound)]

# Comprobar nuevo tamaño del dataset
print("Original:", df.shape)
print("Sin outliers:", df_no_outliers.shape)

1. Violin Plots:

- Use a violin plot to visualize the distribution of price across different product categories. Filter out the top 20 categories based on count for better visualization.

- Which product category tends to have the highest median price? Don't filter here by top categories.


In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# Top 20 categorías con más productos
top_20_cats = df_no_outliers['category'].value_counts().head(20).index
df_violin = df_no_outliers[df_no_outliers['category'].isin(top_20_cats)]

plt.figure(figsize=(15, 8))
sns.violinplot(data=df_violin, x='category', y='price', inner='box', palette="Set3")
plt.xticks(rotation=45, ha='right')
plt.title('Distribución de precios por categoría (Top 20 categorías)')
plt.tight_layout()
plt.show()


In [ ]:
df_no_outliers.groupby('category')['price'].median().sort_values(ascending=False).head()


2. Bar Charts:

- Create a bar chart comparing the average price of products for the top 10 product categories (based on count).
- Which product category commands the highest average price? Don't filter here by top categories.

In [ ]:
# Calcular precio medio por categoría
top_10_cats = df_no_outliers['category'].value_counts().head(10).index
df_bar = df_no_outliers[df_no_outliers['category'].isin(top_10_cats)]

avg_prices = df_bar.groupby('category')['price'].mean().sort_values(ascending=False)

plt.figure(figsize=(12,6))
sns.barplot(x=avg_prices.index, y=avg_prices.values, palette="muted")
plt.ylabel("Precio promedio")
plt.title("Precio promedio por categoría (Top 10 más comunes)")
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.show()

In [ ]:
df_no_outliers.groupby('category')['price'].mean().sort_values(ascending=False).head()

3. Box Plots:

- Visualize the distribution of product ratings based on their category using side-by-side box plots. Filter out the top 10 categories based on count for better visualization.

- Which category tends to receive the highest median rating from customers? Don't filter here by top categories.

In [ ]:
# Top 10 categorías con más productos
top_10_rating_cats = df_no_outliers['category'].value_counts().head(10).index
df_box = df_no_outliers[df_no_outliers['category'].isin(top_10_rating_cats)]

plt.figure(figsize=(14, 6))
sns.boxplot(data=df_box, x='category', y='stars', palette='coolwarm')
plt.xticks(rotation=45, ha='right')
plt.title("Distribución de ratings por categoría (Top 10)")
plt.tight_layout()
plt.show()


In [ ]:
df_no_outliers.groupby('category')['stars'].median().sort_values(ascending=False).head()

 ## Part 3: Investigating the Interplay Between Product Prices and Ratings

Objective: Analyze how product ratings (stars) correlate with product prices.

1. Correlation Coefficients:
- Calculate the correlation coefficient between price and stars.
- Is there a significant correlation between product price and its rating?

In [ ]:
# Calculating the Pearson correlation coefficient between 'stars' and 'price'columns
correlation = df['stars'].corr(df['price'])
correlation

No hay una relación fuerte entre estas variables.

In [ ]:
# Calculating the Spearman rank correlation 
correlation = df['stars'].corr(df['price'], method='spearman')
correlation

LOS PRODUCTOS MÁS CAROS NO TIENDEN A RECIBIR MEJORES NI PEORES RATINS, NI LINEAL NI MONOTÓNICAMENTE.

2. Visualizations:
- Use a scatter plot to visualize the relationship between product rating and price. What patterns can you observe?
- Use a correlation heatmap to visualize correlations between all numerical variables.
- Examine if product prices typically follow a normal distribution using a QQ plot.

In [ ]:
#1. Scatter plot: Precio vs Rating (stars)

import matplotlib.pyplot as plt
import seaborn as sns

plt.figure(figsize=(10, 6))
sns.scatterplot(data=df, x='stars', y='price', alpha=0.5, color='teal')
plt.title('Relación entre rating (stars) y precio')
plt.xlabel('Rating (stars)')
plt.ylabel('Precio')
plt.grid(True)
plt.show()

#Scatter plot: puedes detectar si los productos más caros tienden a tener más o menos estrellas, 
#o si no hay una relación clara.

In [ ]:
#2. Heatmap de correlación entre variables numéricas
plt.figure(figsize=(10, 8))
correlation_matrix = df.select_dtypes(include='number').corr()

sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', fmt=".2f", square=True)
plt.title('Mapa de correlación entre variables numéricas')
plt.show()

#Heatmap: muestra qué variables numéricas están correlacionadas (por ejemplo, price y stars).

In [ ]:
# 3. QQ Plot para ver si price sigue distribución normal
import scipy.stats as stats
import matplotlib.pyplot as plt

plt.figure(figsize=(6, 6))
stats.probplot(df_no_outliers['price'], dist="norm", plot=plt)
plt.title('QQ Plot para Precio')
plt.grid(True)
plt.show()

#QQ Plot: si los puntos siguen la línea diagonal, la distribución es aproximadamente normal. 
# #Si se desvían mucho, no lo es (lo más probable, debido a outliers o sesgo).
